## FEATS-sep
##### Warning: this is gonna be long!<br/>

In [1]:
from corpuscula.corpus_utils import download_ud, UniversalDependencies, \
                                    AdjustedForSpeech
import junky
from mordl.feats_tagger import FeatsSeparateTagger

BERT_MODEL_FN = 'bert-base-multilingual-cased'
MODEL_FN = 'feats_model'
SEED=42

# we use UD Taiga corpus only as example. For real model training comment
# Taiga and uncomment SynTagRus
#corpus_name = 'UD_Russian-Taiga'
corpus_name = 'UD_Russian-SynTagRus'
download_ud(corpus_name, overwrite=False)
corpus = UniversalDependencies(corpus_name)
#corpus = AdjustedForSpeech(corpus)

done: 9525 bytes


In [2]:
tagger = FeatsSeparateTagger()
tagger.load_train_corpus(corpus.train)
tagger.load_test_corpus(corpus.dev)
#tagger.remove_rare_feats(rel_thresh=.1)

_ = tagger.train(MODEL_FN, device='cuda:8', word_emb_type='bert',
                 word_emb_path_suffix=BERT_MODEL_FN
                                     + '_len512_ep3_bat8_seed42',
                 word_emb_tune_params={
                     'model_name': BERT_MODEL_FN,
                     'max_len': 512, 'epochs': 3, 'batch_size': 8
                 },
                 rnn_emb_dim=None, cnn_emb_dim=None, cnn_kernels=range(1, 7),
                 upos_emb_dim=200, lstm_layers=3, seed=SEED)

Train: Load corpus
[=================================================] 48814           
Corpus has been loaded: 48814 sentences, 871526 tokens
Test: Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


###### FEATS TAGGER TRAINING PIPELINE ######

We're gonna train separate models for all FEATS in train corpus. Feats are:

Animacy, Aspect, Case, Degree, Foreign, Gender, Mood, Number, Person, Polarity, Tense, Variant, VerbForm, Voice

=== FEATS:Animacy TAGGER TRAINING PIPELINE ===

BERT MODEL TUNING 'bert-base-multilingual-cased'. The result's model name will be 'feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42'
Loading tokenizer...
Tokenizer is loaded. Vocab size: 119547
Corpora processing... done.
Loading model 'bert-base-multilingual-cased'... done.
Epoch 1: 100%|██████████| 48814/48814 [12:28<00:00, 65.23it/s, train_loss=0.135]   
Average train loss: 0.06171073812945936


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Dev: accuracy = 0.98730239
Dev: precision = 0.73395393
Dev: recall = 0.72911685
Dev: f1_score = 0.73149664
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 48814/48814 [12:20<00:00, 57.70it/s, train_loss=0.00175] 
Average train loss: 0.024529414104471663
Dev: accuracy = 0.98909387
Dev: precision = 0.98026741
Dev: recall = 0.97831461
Dev: f1_score = 0.97928765
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 48814/48814 [12:14<00:00, 66.50it/s, train_loss=0.000714]
Average train loss: 0.01058906736019653
Dev: accuracy = 0.98940833
Dev: precision = 0.98085384
Dev: recall = 0.97852986
Dev: f1_score = 0.97968497
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Dev: accuracy = 0.97969812
Dev: precision = 0.87085707
Dev: recall = 0.83399896
Dev: f1_score = 0.84918865
NB: Scores may be high because of tags stretching
Saving model to feats-case_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 48814/48814 [12:07<00:00, 67.14it/s, train_loss=0.0433]  
Average train loss: 0.037854013327182814
Dev: accuracy = 0.98454370
Dev: precision = 0.85465604
Dev: recall = 0.85961626
Dev: f1_score = 0.85703655
NB: Scores may be high because of tags stretching
Saving model to feats-case_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 48814/48814 [12:01<00:00, 67.68it/s, train_loss=0.000423]
Average train loss: 0.01834296936928189
Dev: accuracy = 0.98556332
Dev: precision = 0.82348714
Dev: recall = 0.87503553
Dev: f1_score = 0.84136787
NB: Scores may be high because of tags stretching
Saving model to feats-case_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%|████████

In [3]:
res_dev = 'corpora/_dev_' + MODEL_FN.replace('-sep_model', '.conllu')
res_test = 'corpora/_test_' + MODEL_FN.replace('-sep_model', '.conllu')
tagger = FeatsSeparateTagger()
tagger.load(MODEL_FN)
junky.clear_tqdm()

### Loading FEATS tagger:

--- Animacy:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Aspect:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Case:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Degree:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Foreign:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Gender:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Mood:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Number:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.

--- Person:
Loading dataset... done.
Creating model... 

In [4]:
_ = tagger.predict(corpus.dev, clone_ds=True, save_to=res_dev)

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.67it/s]
Adjusting
100%|██████████| 6584/6584 [00:24<00:00, 271.75it/s]
Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.63it/s]
Adjusting
100%|██████████| 6584/6584 [00:25<00:00, 259.38it/s]
Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.62it/s]
Adjusting
100%|██████████| 6584/6584 [00:23<00:00, 274.74it/s]
Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.61it/s]
Adjusting
100%|██████████| 6584/6584 [00:25<00:00, 255.11it/s]
Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.54it/s]
Adjusting
100%|██████████| 6584/6584 [00:26<00:00, 247.83it/s]
Vectorizing
100%|██████████| 103/103 [00:12<00:00, 15.13it/s]
Adjusting
100%|██████████| 6584/6584 [00:29<00:00, 222.68it/s]
Vectorizing
100%|██████████| 103/103 [00:15<00:00,  6.79it/s]
Adjusting
100%|██████████| 6584/6584 [00:28<00:00, 228.76it/s]
Vectorizing
100%|██████████| 103/103 [00:15<00:00, 14.43it/s]
Adjusting
100%|██████████| 6584/6584 [00:26<00:00, 252.07it/s]


In [5]:
_ = tagger.evaluate(corpus.dev)

Evaluating FEATS


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Processing corpus
100%|██████████| 6584/6584 [01:18<00:00, 83.96it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:12<00:00, 90.64it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:12<00:00, 90.43it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:11<00:00, 91.66it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:11<00:00, 92.68it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:11<00:00, 92.29it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:09<00:00, 94.19it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:10<00:00, 93.78it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:09<00:00, 94.77it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:09<00:00, 94.53it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:09<00:00, 94.30it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:12<00:00, 90.84it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:11<00:00, 91.52it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:11<00:00, 92.64it/s] 
FEATS 

In [6]:
_ = tagger.evaluate(corpus.dev, res_dev)

Evaluating FEATS
Load corpus
[> 400                                                            

Load corpus
[> 400                                                            

[> 900                                                            

[> 900                                                            

[=> 1400                                                           

...

[=====> 5200                                                       

[=====> 5700                                                       

[=====> 5800                                                       

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
FEATS total: 118488 tokens, 287286 tags
    correct: 116019 tokens, 283958 tags
      wrong: 2469 tokens, 3328 tags [611 excess / 562 absent / 2155 wrong type]
   Accuracy: 0.9661562924074404 / 0.9884157250962455
[Total accuracy: 0.9791624468300587 / 0.9884157250962455]


[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [7]:
_ = tagger.predict(corpus.test, save_to=res_test)

Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Processing corpus
100%|██████████| 6491/6491 [01:09<00:00, 93.29it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 92.55it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 91.86it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:09<00:00, 92.95it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:11<00:00, 91.27it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:09<00:00, 93.14it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:09<00:00, 92.93it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:12<00:00, 89.02it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 92.31it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 91.58it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 92.04it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:09<00:00, 92.80it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:10<00:00, 91.50it/s] 
Processing corpus
100%|██████████| 6491/6491 [01:08<00:00, 95.05it/s] 


In [8]:
_ = tagger.evaluate(corpus.test, clone_ds=True)

Evaluating FEATS


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.74it/s]
Adjusting
100%|██████████| 6491/6491 [00:23<00:00, 272.63it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.72it/s]
Adjusting
100%|██████████| 6491/6491 [00:24<00:00, 262.23it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.72it/s]
Adjusting
100%|██████████| 6491/6491 [00:23<00:00, 273.16it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.76it/s]
Adjusting
100%|██████████| 6491/6491 [00:23<00:00, 272.74it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.70it/s]
Adjusting
100%|██████████| 6491/6491 [00:24<00:00, 269.30it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.66it/s]
Adjusting
100%|██████████| 6491/6491 [00:24<00:00, 267.15it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.63it/s]
Adjusting
100%|██████████| 6491/6491 [00:24<00:00, 262.92it/s]
Vectorizing
100%|██████████| 102/102 [00:10<00:00,  9.67it/s]
Adjusting
100%|██████████| 6491/6491 [00:24<00:00, 263.58it/s]


In [9]:
_ = tagger.evaluate(corpus.test, res_test)

Evaluating FEATS
Load corpus
[> 600                                                            

Load corpus
[> 600                                                            

[=> 1500                                                           

[=> 1500                                                           

[==> 2300                                                          

...

[=====> 5700                                                       

[======> 6200                                                      

[======> 6300                                                      

[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens
FEATS total: 117329 tokens, 285311 tags
    correct: 114837 tokens, 281860 tags
      wrong: 2492 tokens, 3451 tags [547 excess / 743 absent / 2161 wrong type]
   Accuracy: 0.9655691725271841 / 0.9879044270988501
[Total accuracy: 0.9787605792259373 / 0.9879044270988501]


[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


In [10]:
corp_gold = list(corpus.test())
corp_test = list(tagger._get_corpus(res_test))
tags = sorted(set(x for x in corp_gold
                    for x in x[0]
                    for x in x['FEATS'].keys()))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp_gold, corp_test,
                             feats=tag, log_file=None)
    ))

Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens
Animacy: 0.9831625183016105
Aspect: 0.9934310257705912
Case: 0.9818084537185661
Degree: 0.9966804979253112
Foreign: 0.7349397590361446
Gender: 0.9826337751595483
Mood: 0.994475138121547
Number: 0.9922937890239895
Person: 0.9941728254690165
Polarity: 0.8913043478260869
Tense: 0.9935737236701179
Variant: 0.9845105328376704
VerbForm: 0.998412354766544
Voice: 0.9903991915108641
